# RNNs

Y'all punks can get the data from kaggle here: https://www.kaggle.com/uciml/sms-spam-collection-dataset/kernels 

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/spam.csv', encoding = 'latin1')

In [2]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
from html import parser

# Drop the empty columns
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

# Rename the columns
df = df.rename(index=str, columns={"v1": "classification", "v2": "message"})

# Remove HTML encodings
df['message'] = df['message'].apply(parser.unescape)
df.head()

,classification,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['classification'] = pd.get_dummies(df['classification'])

In [5]:
print(df.head())

   classification                                            message
0               1  Go until jurong point, crazy.. Available only ...
1               1                      Ok lar... Joking wif u oni...
2               0  Free entry in 2 a wkly comp to win FA Cup fina...
3               1  U dun say so early hor... U c already then say...
4               1  Nah I don't think he goes to usf, he lives aro...


In [6]:
y = np.array(df['classification'].values.astype(np.int))
np.unique(y)

array([0, 1])

In [7]:
import keras


y = np.array(df['classification'].values)
X = df['message'].values
#print(X)

#y = [0 if y_ == "spam" else 1 for y_ in y]
y_ohe = keras.utils.to_categorical(y)
y_ohe


Using TensorFlow backend.


array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

We need to find how long our longest message is

In [8]:
max = 0
count = 0
for i in X:
    if len(i) > max:
        #if len(i) < 200:
        max = len(i)
        #else:
            #count +=1
        
print(max)
print(count)

910
0


In [9]:
from keras.preprocessing.text import Tokenizer
NUM_TOP_WORDS = None

tokenizer = Tokenizer(num_words=NUM_TOP_WORDS)
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

In [10]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_ohe, test_size=0.2)


# truncate and pad input sequences to be the same length
max_text_length = 910
X_train = sequence.pad_sequences(X_train, maxlen=max_text_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_text_length)

NUM_CLASSES = 2

In [11]:
from keras.layers import Embedding
EMBED_SIZE = 100


word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, EMBED_SIZE))
embedding_layer = Embedding(len(word_index) + 1,
                            EMBED_SIZE,
                            weights=[embedding_matrix],
                            input_length=max_text_length,
                            trainable=False)

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

rnn = Sequential()
rnn.add(embedding_layer)
rnn.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
rnn.add(Dense(NUM_CLASSES, activation='sigmoid'))
rnn.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])
print(rnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 910, 100)          892000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 972,602
Trainable params: 80,602
Non-trainable params: 892,000
_________________________________________________________________
None


In [13]:
rnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64)

Train on 4457 samples, validate on 1115 samples
Epoch 1/1
4457/4457 [==============================] - 93s - loss: 0.4104 - acc: 0.8643 - val_loss: 0.4145 - val_acc: 0.8565


In [14]:
yhat = np.argmax(rnn.predict(X_test), axis=1)

In [15]:
for a in yhat:
    print(a)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [16]:
from sklearn import metrics as mt
rec = mt.recall_score(y_test,yhat.round())
acc = mt.accuracy_score(y_test,yhat.round())
print(rec)
print(acc)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [ ]:
np.unique(yhat)
np.unique(y_test)